In this notebook, we show the dependency between the proportion of mixed MAP states and the number of edges M in a random (N, M)-graph

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from opt import *

def random_ferromagnet(G, seed=0):
    np.random.seed(seed)

    J = nx.linalg.graphmatrix.adjacency_matrix(G).todense()
    H = np.random.rand(len(J), len(J))
    J = np.multiply(J, (H + H.T) / 2)
    mu = 10 * np.random.rand()
    J *= mu
    h = [-1 for _ in range(len(J))]

    return J, h

def get_portions(n, samples=100, step=0.05):
    portions = []
    probs = []
    p = step
    while p < 1.0:
        m_intermediate = 0.0
        for s in range(samples):
            G = nx.gnp_random_graph(n, p, seed=s)
            J, h = random_ferromagnet(G, seed=s)
            I = np.random.choice(list(range(n)), 1)
            result = gurobi_map_explicit(J, h, I)

            if (sum(result) != n) and (sum(result) != 1):
                m_intermediate += 1.0

        portions.append(m_intermediate/samples)
        probs.append(p)
        p += step

    return probs, portions

def get_portions_m(n, mlist, samples=500):
    portions = []
    for m in mlist:
        m_intermediate = 0.0
        for s in range(samples):
            G = nx.gnm_random_graph(n, m, seed=s)
            J, h = random_ferromagnet(G, seed=s)
            I = np.random.choice(list(range(n)), 1)
            result = gurobi_map_explicit(J, h, I)

            if (sum(result) != n) and (sum(result) != 1):
                m_intermediate += 1.0

        portions.append(m_intermediate / samples)

    return portions

mlist = list(range(5, 105, 2))
s = 500
portions = get_portions_m(10, mlist, samples=s)
plt.plot(mlist, portions, 'g', label='N = 10')

portions = get_portions_m(15, mlist, samples=s)
plt.plot(mlist, portions, 'm', label='N = 15')

portions = get_portions_m(20, mlist, samples=s)
plt.plot(mlist, portions, 'b', label='N = 20')

portions = get_portions_m(30, mlist, samples=s)
plt.plot(mlist, portions, 'y', label='N = 30')

plt.legend()
plt.xlabel('Number of edges in a random (N, M)-graph')
plt.ylabel('Proportion of mixed MAP states')
plt.savefig('density_m_500.png', dpi=300)